In [1]:
#!pip install torchvision

#!pip install torch


In [2]:
from __future__ import print_function, division
from __future__ import print_function
from __future__ import division

import torch
import torch.nn as nn

import torch.optim as optim

from torch.optim import lr_scheduler



import torchvision
from torchvision import datasets, models, transforms

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time

import os
import copy
import shutil


plt.ion() #interactive mode

from torchvision.models import ResNet50_Weights, ResNet18_Weights



In [3]:
#os.listdir()

#training_data='./Data/Throughput/A_folderize/train'
input_path = './Data/Input/'


input_train = input_path + 'train_set/train_set/'
input_test = input_path + 'test_set/test_set/test_set/'
input_labels_test = input_path + 'sample.csv'
input_labels_train = input_path + 'train_labels.csv'
    
throughput_path = './Data/Throughput/'
A_trainset = throughput_path + 'A_folderize/train/'
A_testset = throughput_path + 'A_folderize/test/'
A_validationset = os.path.join(throughput_path, 'A_folderize/train')
save_model='./save_model'

In [4]:
# # a_folderize
# def make_folders():
#     for path in [A_trainset, A_testset]: #create directories
#         if not os.path.exists(path):
#             os.mkdir(path)
# def copy_file(source, destination):
#     shutil.copyfile(source, destination)
    
# def A_Folderize(force = True):
    
#     """Creates folders that are necessary in general and for training data.

#     This function creates folders that are necessary in general and for training
#     data, and then puts pictures in the newly created folders. The `force`
#     parameter is optional and is set to `False` by default. If set to `True`,
#     the function will re-create the folders even if they already exist.

#     Args:
#         force: bool, optional
#             If set to `True`, the function will re-create the folders even if
#             they already exist.

#     Returns:
#         None
#     """
#     print('START: A_folderize')
#     print('Creating folders that are necessary in general')
#     make_folders()

#     labels_test = pd.read_csv(input_labels_test)
#     labels_train = pd.read_csv(input_labels_train)

#     print('Creating folders that are necessary for training data')
#     pathh = os.path.join(A_trainset, '0')
#     if (not os.path.exists(pathh)) | force:
#         for label in labels_train['label'].unique():
#             path = os.path.join(A_trainset, str(label - 1))
#             os.mkdir(path)
#         for label in labels_test['label'].unique():
#             path = os.path.join(A_testset, str(label - 1))
#             os.mkdir(path)

#         print('Putting pictures in the newly created folders')
#         for picpath in os.listdir(input_train): #picpath is the pathway to one image and the image its img_name,
#             labelfolder = labels_train.loc[labels_train['img_name'] == picpath]['label'].iloc[0] #this returns a number indicating a foodclass
#             copy_file(os.path.join(input_train, picpath),
#                          os.path.join(A_trainset, str(labelfolder - 1), picpath))
#         for picpath in os.listdir(input_test):
#             labelfolder = labels_test.loc[labels_test['img_name'] == picpath]['label'].iloc[0]
#             copy_file(os.path.join(input_test, picpath),
#                          os.path.join(A_testset, str(labelfolder - 1), picpath))

#     print('DONE: A_folderize')


In [5]:
# Set mean and std values for normalization
means = [0.6366, 0.5437, 0.4454]
stds = [0.2235, 0.2422, 0.2654]
def C_PrepareData(input_size):
    """Prepares data for training, validation, and testing.
    This function prepares data for training, validation, and testing by
    applying transformations to the images in the datasets and loading the
    transformed images into data loaders.
    Args:
        input_size: int
            The input size of the model.
    Returns:
        train_loader: the data loader for the training dataset
        val_loader: the data loader for the validation dataset
        test_loader: the data loader for the testing dataset
    """

    print('START: C_PrepareData')
    # resize, pixels. resize depending on data, we have to explore other sizes to check for performance
    training_transforms = transforms.Compose([transforms.Resize((input_size, input_size)), transforms.ToTensor()])
    print(training_transforms)

    train_dataset = torchvision.datasets.ImageFolder(root=A_trainset, transform=training_transforms)
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=32, shuffle=False)

    # image data preparation
    # normalized data=> image=(image-mean)/std
    # Data augmentation and normalization for training
    # Just normalization for validation
    data_transforms = {
        'train': transforms.Compose([
            transforms.RandomResizedCrop(input_size), # TODO: Check if better crop or transforms.Resize((input_size, input_size)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(15),
            transforms.ToTensor(),
            transforms.Normalize(means,stds)
        ]),
        'val': transforms.Compose([
            transforms.Resize(input_size), # TODO: Check if resizing helps or not
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            transforms.Normalize(means, stds)
        ])
    }

    data_transforms['test'] = data_transforms['val']

    train_dataset = torchvision.datasets.ImageFolder(root=A_trainset, transform=data_transforms['train'])
    val_dataset = torchvision.datasets.ImageFolder(root=A_validationset, transform=data_transforms['val'])
    test_dataset = torchvision.datasets.ImageFolder(root=A_testset, transform=data_transforms['test'])

    # Mini-Batch Gradient Descent, start with 32 and explore to increase performance
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True)
    print('DONE: C_PrepareData')

    return train_loader, val_loader, test_loader


In [6]:
#C_PrepareData(3)

In [7]:
#Hyperparameters

# Set number of training epochs and batch size
num_epochs = 15
batch_size = 16

# Set learning rate and momentum
learning_rate = 0.009
momentum = 0.11

# Flag for feature extracting. When False, we finetune the whole model,
# when True we only update the reshaped layer params
feature_extract = True


In [8]:
def define_model(model):
    """Modify the fully-connected layer of a PyTorch model to match the number of features and classes in the input data.
    Args:
        model (torch.nn.Module): The PyTorch model to modify.
    Returns:
        torch.nn.Module: The modified PyTorch model.
    """
    num_fts = model.fc.in_features
    number_of_classes = len(pd.read_csv(paths.input_labels_train)['label'].unique())

    model.fc = nn.Linear(num_fts, number_of_classes)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    return model

def init_optimizer(model_ft, device, feature_extract = True):
    # Send the model to GPU
    model_ft = model_ft.to(device)

    # Gather the parameters to be optimized/updated in this run. If we are
    #  finetuning we will be updating all parameters. However, if we are
    #  doing feature extract method, we will only update the parameters
    #  that we have just initialized, i.e. the parameters with requires_grad
    #  is True.
    params_to_update = model_ft.parameters()
    print("Params to learn:")
    if feature_extract:
        params_to_update = []
        for name, param in model_ft.named_parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
                print("\t", name)
    else:
        for name, param in model_ft.named_parameters():
            if param.requires_grad == True:
                print("\t", name)

    # Observe that all parameters are being optimized
    optimizer_ft = optim.SGD(params_to_update, lr=0.01, momentum=0.11)
    return optimizer_ft


def train_nn(model, optimizer, train_loader, test_loader, criterion,
             n_epochs):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    

    for epoch in range(n_epochs):
        model.train()
        running_loss = 0.0
        running_correct = 0.0
        total = 0
#validation
        for data in train_loader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            total += labels.size(0)

            optimizer.zero_grad()

            outputs = model(images)

            _, predicted = torch.max(outputs.data, 1)

            loss = criterion(outputs, labels)

            loss.backward()

            optimizer.step()

            running_loss += loss.item()
            running_correct += (labels == predicted).sum().item()

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = running_correct / total

        print(f"Epoch {epoch + 1}: Correct: {running_correct} ({epoch_acc}%); Loss: {epoch_loss}")



In [9]:


def train_model(model, train_loader, val_loader, num_epochs=num_epochs, is_inception=False):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    since = time.time()
    val_acc_history = []
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    # Print the model we just instantiated
    print(f"Model: {model}")

    # Send the model to GPU
    model = model.to(device)

    # Gather the parameters to be optimized/updated in this run. If we are
    #  finetuning we will be updating all parameters. However, if we are
    #  doing feature extract method, we will only update the parameters
    #  that we have just initialized, i.e. the parameters with requires_grad
    #  is True.
    params_to_update = model.parameters()
    print("Params to learn:")
    if feature_extract:
        params_to_update = []
        for name,param in model.named_parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
                print("\t",name)
    else:
        for name,param in model.named_parameters():
            if param.requires_grad == True:
                print("\t",name)

    # Observe that all parameters are being optimized
    optimizer = optim.SGD(params_to_update, lr=learning_rate, momentum=momentum)

    # Setup the loss fxn
    criterion = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            dataloaders = {
                'train': train_loader,
                'val': val_loader
            }
            for data in dataloaders[phase]:
                inputs, labels = data
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history





In [10]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet18":
        """ Resnet18
        """
        model_ft = models.resnet18(weights=ResNet18_Weights.DEFAULT)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "resnet50":
        """ Resnet50
        """
        model_ft = models.resnet50(weights=ResNet50_Weights.DEFAULT)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        Exception("Invalid model name, exiting. To change model name, change the model_name variable in Config.ConfigMain")
        exit()

    return model_ft, input_size

model_name = "resnet50"
feature_extract=True




def B_InitModel():
    """Initializes a model.
    This function initializes a model and returns the initialized model and its
    input size.
    Returns:
        model: the initialized model
        input_size: the input size of the model
    """
    print('START: B_InitModel')
    num_classes = len(os.listdir(A_trainset))
    model, input_size = initialize_model(model_name, num_classes, feature_extract)
    print('DONE: B_InitModel')
    return model, input_size
#B_InitModel()

In [11]:
def D_TrainModel(model, train_loader, test_loader):
    print('START: D_TrainModel')
    model = train_model(model, train_loader, test_loader, num_epochs=num_epochs, is_inception=False)
    print('DONE: D_TrainModel')
    return model



In [12]:

# def predict(model, test_loader): 
#     predictions = []
#     model.eval() #set to evaluate, which might impact how the calculations are done
#     with torch.no_grad(): #no gradient needs to be computed for evaluation
#         for x, y in test_loader:
#             x = x.reshape(x.shape[0], -1)

#             score = model(x)
#             predictions.append(score)

#     """ 
#   predictions = []
#   for item in test_loader:
#     print (item)
#     with torch.no_grad():
#         # Iterate over data.
#         for inputs, _ in test_loader:
#             outputs = model(inputs)
#             _, preds = torch.max(outputs, 1)
#         predictions.extend(preds.numpy())
#     """
#     return predictions

def E_PredictModel(model, test_loader):
    print("Predictions")
    model.eval()
    #with torch.no_grad():
    imagenames = []

    for data in test_loader.dataset.imgs:
        imgname = data[0].split("/")[-1]
        imgname = imgname[2:]
        imagenames.append(imgname)
        break
    
    

    predictions = []
    for item_image, _ in test_loader.dataset:
        current_image = torch.unsqueeze(item_image, 0)
        perhaps_image_name, prediction_class = torch.max(model(current_image), 1)
        this_prediction = prediction_class[0]
        print(perhaps_image_name, int(this_prediction))
        predictions.append(int(this_prediction))
        break
    print(predictions)

    df = pd.DataFrame({
        "img_name" : imagenames,
        "label" : predictions
    })
    print(df)
    df.to_csv("test_van_resnet.csv", index=False)


In [ ]:
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

def main(*pipeparts):
    if 'A' in pipeparts:
        A_Folderize(force=False)
    if 'B' in pipeparts:
        model, input_size = B_InitModel()
    if 'C' in pipeparts:
        train_loader, val_loader, test_loader = C_PrepareData(input_size)
    if 'D' in pipeparts:
        model = D_TrainModel(model, train_loader, val_loader)
        # Save the model's state_dict
        torch.save(model.state_dict(), save_model)
        print('Saved Pytorch model state')
        model.load_state_dict(torch.load(save_model))
    if 'E' in pipeparts:
        predictions = pipe.E_PredictModel(model, test_loader)

    return predictions
    
        
       


if __name__ == '__main__':
    # Define which parts of the pipeline to execute (include 'A' in first execution, then 'A' does not need to be run)
    pipeparts = ['B', 'C', 'D','E']
    main(*pipeparts)





PyTorch Version:  1.13.0+cpu
Torchvision Version:  0.14.0+cpu
START: B_InitModel
DONE: B_InitModel
START: C_PrepareData
Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
    ToTensor()
)
DONE: C_PrepareData
START: D_TrainModel
Model: ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d

train Loss: 4.1602 Acc: 0.1324
val Loss: 3.8872 Acc: 0.2477

Epoch 1/14
----------
train Loss: 3.7876 Acc: 0.2374
val Loss: 3.5169 Acc: 0.3083

Epoch 2/14
----------
train Loss: 3.5153 Acc: 0.2773
val Loss: 3.2220 Acc: 0.3501

Epoch 3/14
----------
train Loss: 3.3128 Acc: 0.3108
val Loss: 3.0176 Acc: 0.3818

Epoch 4/14
----------
train Loss: 3.1592 Acc: 0.3279


In [ ]:
start:around 2

In [ ]:
# def D_EvaluateModel():

# with torch.no_grad():
#     correct = 0
#     total = 0
#     for images, labels in test_loader:
#         images = images.to(device)
#         labels = labels.to(device)
#         outputs = model(images)
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()
#         del images, labels, outputs

#     print('Accuracy of the network on the {} test images: {} %'.format(10000, 100 * correct / total))   
    
#     #same as validation but with test_loader

In [ ]:
predict()

In [ ]:
def plot_model():
    # TODO: Not finished yet
    # Plot the training curves of validation accuracy vs. number
    #  of training epochs for the transfer learning method and
    #  the model trained from scratch
    hist = []
    ohist = [h.cpu().numpy() for h in hist]
    plt.title("Validation Accuracy vs. Number of Training Epochs")
    plt.xlabel("Training Epochs")
    plt.ylabel("Validation Accuracy")
    plt.plot(range(1,num_epochs+1),ohist,label="Pretrained")
    plt.ylim((0,1.))
    plt.xticks(np.arange(1, num_epochs+1, 1.0))
    plt.legend()
    plt.show()

In [ ]:

# def get_mean_and_std(loader):
#     mean=0.
#     std=0.
#     total_images_count=0
#     for images,_ in loader:
#         image_count_in_a_batch=images.size(0)
#         images=images.view(image_count_in_a_batch, images.size(1), -1)
#         mean+=images.mean(2).sum(0)
#         std+=images.std(2).sum(0)
#         total_images_count+=image_count_in_a_batch
#     mean/=total_images_count
#     std/=total_images_count
#     return mean,std
    

In [ ]:
 #get_mean_and_std(train_loader)

In [ ]:
# def show_transf_images(data):
#     loader=torch.utils.data.DataLoader(data,batch_size=6, shuffle=True)
#     batch=next(iter(loader))
#     images,labels=batch
    
#     grid=torchvision.utils.make_grid(images,nrow=3)
#     plt.figure(figsize=(11,11))
#     plt.imshow(np.transpose(grid,(1,2,0)))
#     print('labels:',labels)
    

In [ ]:
#show_transf_images(test_dataset)

